# Задание 4

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re

## Загрузка DataFrame

### Задача 1

Создаем датафрейм и загружаем данные по ссылке, одновременно заменяя стандартный индекс на колонку Показатель

In [2]:
url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('UTF8')),index_col=0)

Добавляем столбец соответствия нормативу, по умолчанию ставим несоответствие, проверяем, что получилось

In [3]:
df["Соответствие норме"] = 'Нет'
df

,Единица измерений,Результат анализа,Норматив,Соответствие норме
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Нет
Запах,баллы,1,не более 2-3,Нет
Цветность,градусы,б/цвета,не более 30,Нет
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Нет
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Нет
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Нет
Нитраты (по NO3),мг/дм3,24,не более 45,Нет
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Нет
Хлориды (Cl),мг/дм3,200,не более 350,Нет


Считываем индексы (т. е. показатели)

In [4]:
index=df.index

По каждой строке парсим колонку Норматив и превращаем строку в значения.
Если результат соотвествует значениям - устанавливаем соответствие

In [5]:
for i in index:
    s=df.loc[i][2]
    ## В данном случае норматив бывает только двух видов, поэтому анализируем две строковые константы
    if (s.split()[0]+' '+s.split()[1]) == 'в пределах':
        s1=s.split()[2]
        min=float(s1.split("-")[0])
        max=float(s1.split("-")[1])
        val=float(df.loc[i][1])
        if min <= val <= max: 
            df['Соответствие норме'][i] = 'Да'
    elif (s.split()[0]+' '+s.split()[1]) == 'не более':
        s1=s.split()[2]
        max=float(s1.replace(',', '.').split("-")[0]) ##при парсинге заменим запятую на точку, иначе не сможем получить число
        if df.loc[i][1] == 'б/цвета':
            val=0; ##предполагаем, что без цвета - это нулевой градус цветности
        else:
            val=float(df.loc[i][1])
        if val <= max: 
            df['Соответствие норме'][i] = 'Да'
    else:
        print (s,'Такой тип норматива не обрабатывается') ## на всякий случай исключение

Проверям, что получилось

In [6]:
df

,Единица измерений,Результат анализа,Норматив,Соответствие норме
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Да
Запах,баллы,1,не более 2-3,Да
Цветность,градусы,б/цвета,не более 30,Да
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Да
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Да
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Да
Нитраты (по NO3),мг/дм3,24,не более 45,Да
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Да
Хлориды (Cl),мг/дм3,200,не более 350,Да


По двум показателям норматив не пройден

## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2

В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?

Создаем ящик как массив с заданными фркутами

In [7]:
box = np.array(['А','А','А','А','А','Я','Я','Я','Я'])

Проверяем созданный массив, все корректно

In [8]:
box

array(['А', 'А', 'А', 'А', 'А', 'Я', 'Я', 'Я', 'Я'], dtype='<U1')

Делаем 100 попыток случайно выбрать 3 элемента из ящика, подсчитываем число выпадений трех апельсинов

In [42]:
count = 0
for i in range(100):
    test = np.random.choice(box, size = 3, replace = False)
    if np.count_nonzero(test == "А") == 3:
        count += 1
print (count)



10


Попробуем сделать выборку 10 тыс раз

In [43]:
count = 0
for i in range(10000):
    test = np.random.choice(box, size = 3, replace = False)
    if np.count_nonzero(test == "А") == 3:
        count += 1
print (count)


1233


Видим, что экспериметально вероятность примерно равна теоретической

### Задача 3

Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?

В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

Создаем массив деталей - 7 стандартных, 3 нестандартных

In [45]:
details = np.array(['С','С','Н','С','С','С','Н','С','С','Н'])

Делаем случайную выборку двух деталей, проверяем, что первая нестандартная, а вторая - стандартная

In [70]:
count = 0
for i in range(10000):
    set = np.random.choice(details, size = 2, replace = False)
    if set[0] == 'Н' and set[1] == 'С':
        count += 1
print (count)

2335


При 10 тыс. попыток экспериментальное значение практически точно совпадает с теоретическим.